In [2]:
from langchain_core.prompts import ChatPromptTemplate,MessagesPlaceholder

prompt = ChatPromptTemplate.from_messages(
    [
        ("system","You are a helpful assistant. Answer all the questions to the best of your ability."),
        MessagesPlaceholder(variable_name="messages")
    ]
)

import os
from dotenv import load_dotenv
load_dotenv() # load all env variables

groq_api_key = os.getenv("GROQ_API_KEY")
groq_api_key

from langchain_groq import ChatGroq
model = ChatGroq(model = "Gemma2-9b-It",groq_api_key=groq_api_key)
model

chain = prompt|model

In [3]:
from langchain_core.messages import HumanMessage

chain.invoke({"messages":[HumanMessage(content = "Hi, My name is Deepak Rai.")]})

AIMessage(content="Hi Deepak Rai, it's nice to meet you!\n\nWhat can I do for you today? 😊 \n\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 27, 'prompt_tokens': 34, 'total_tokens': 61, 'completion_time': 0.049090909, 'prompt_time': 0.002341025, 'queue_time': 0.262163924, 'total_time': 0.051431934}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run--b1210620-18ba-42f0-ac5e-3b6f11229c5b-0', usage_metadata={'input_tokens': 34, 'output_tokens': 27, 'total_tokens': 61})

In [19]:
# Message History
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

store = {}

def get_session_history(session_id:str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]

with_message_history = RunnableWithMessageHistory(chain, get_session_history)

In [5]:
config = {"configurable":{"session_id": "chat3"}}

response = with_message_history.invoke(
    [HumanMessage(content = "Hi, My name is Deepak Rai.")],
    config=config
)

response

AIMessage(content="Hello Deepak Rai, it's nice to meet you! \n\nHow can I help you today? 😊  \n\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 27, 'prompt_tokens': 34, 'total_tokens': 61, 'completion_time': 0.049090909, 'prompt_time': 0.002349795, 'queue_time': 0.263072965, 'total_time': 0.051440704}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run--2f188091-d4a6-4434-b085-d87241e8c0d8-0', usage_metadata={'input_tokens': 34, 'output_tokens': 27, 'total_tokens': 61})

In [10]:
# Add more complexity to the prompt

prompt = ChatPromptTemplate.from_messages(
    [
        ("system","You are a helpful assistant. Answer all the questions to the best of your ability in {language}"),
        MessagesPlaceholder(variable_name="messages")
    ]
)

chain = prompt|model

In [13]:
response = chain.invoke(
    {
        "messages":[HumanMessage(content = "Hi, My name is Deepak Rai.")],
        "language": "Hindi"
     }
    )

response.content

'नमस्ते Deepak Rai! 😊 \n\nमुझे बहुत खुशी है आपसे मिलने का। आप मुझसे कुछ भी पूछ सकते हैं, मैं अपनी पूरी कोशिश करूँगा कि आपकी मदद करूँ। \n\n'

In [14]:
with_message_history = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages"
)

In [18]:
config = {"configurable":{"session_id": "chat4"}}

response = with_message_history.invoke(

    {"messages": 
     [HumanMessage(content="Hi, My name is Deepak Rai")],
     
     "language":"Hindi"
     },
     config=config
)

response.content

'नमस्ते दीपक राय! 😊 \n\nआप कैसे हैं? क्या मैं आपकी कोई मदद कर सकता हूँ?\n'

In [19]:
response = with_message_history.invoke(

    {"messages": 
     [HumanMessage(content="Hi, What is my name")],
     
     "language":"Hindi"
     },
     config=config
)

response.content

'आपका नाम दीपक राय है।  \n'

In [14]:
from langchain_core.messages import SystemMessage,trim_messages,AIMessage,HumanMessage

trimmer = trim_messages(
    max_tokens=90,
    strategy = "last",
    token_counter = model,
    include_system = True,
    allow_partial = False,
    start_on = "human"
)

messages = [
    SystemMessage(content="you are a good assistant"),
    HumanMessage(content="hi, I am Deepak. Do you remember me?"),
    AIMessage(content="hi!"),
    HumanMessage(content="I like strawberry ice cream"),
    AIMessage(content="nice"),
    HumanMessage(content="What is 2 + 2?"),
    AIMessage(content="4"),
    HumanMessage(content="thanks"),
    AIMessage(content="no problem"),
    HumanMessage(content="having fun?"),
    AIMessage(content="yes!")

]

trimmer.invoke(messages)

[SystemMessage(content='you are a good assistant', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='hi, I am Deepak. Do you remember me?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='hi!', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='I like strawberry ice cream', additional_kwargs={}, response_metadata={}),
 AIMessage(content='nice', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='What is 2 + 2?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='4', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='thanks', additional_kwargs={}, response_metadata={}),
 AIMessage(content='no problem', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='having fun?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='yes!', additional_kwargs={}, response_metadata={})]

In [15]:
from operator import itemgetter

from langchain_core.runnables import RunnablePassthrough

chain = (
    RunnablePassthrough.assign(messages=itemgetter("messages")|trimmer)
    | prompt
    | model
)

response = chain.invoke(
    {
    "messages":messages + [HumanMessage(content="What ice cream do I like?")],
    "language":"English"
    }

)

response.content

'You said you like strawberry ice cream!  🍓🍦\n'

In [16]:
response = chain.invoke(
    {
    "messages":messages + [HumanMessage(content="What math problem did I ask?")],
    "language":"English"
    }

)

response.content

'You asked: What is 2 + 2?\n'

In [ ]:
# let's wrap this in the form of message history

with_message_history = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages"
)

config= {"configurable":{"session_id":"chat5"}}

In [21]:
response = with_message_history.invoke(

    {"messages": 
     [HumanMessage(content="Hi, What is my name?")],
     
     "language":"English"
     },
     config=config
)

response.content

"As an AI, I don't have access to your personal information, including your name.  \n\nIf you'd like to tell me your name, I'd be happy to use it! 😊  \n\n"

In [22]:
response = with_message_history.invoke(

    {"messages": 
     [HumanMessage(content="Hi, What math problem did I ask?")],
     
     "language":"English"
     },
     config=config
)

response.content

"You haven't asked me any math problems yet! \n\nIs there a math problem you'd like to work on together? I'm happy to help. 😄  \n\n"